In [2]:
from selenium import webdriver
# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By
# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點擊、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains
# 加入鍵盤功能 (例如 Ctrl、Alt 等)
from selenium.webdriver.common.keys import Keys
# 強制等待 (執行期間休息一下)
from time import sleep
import requests as req
import os
import ddddocr
from selenium.webdriver.support.ui import Select
import random
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行

my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")
my_options.add_experimental_option("excludeSwitches", ["enable-automation"])
my_options.add_experimental_option('useAutomationExtension', False)



In [3]:
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        });
    """
})
def human_like_pause(min_sec=0.8, max_sec=2.5):
    #模擬人類行為的隨機等待
    sleep(random.uniform(min_sec, max_sec))

def openWebsite():
    # 使用 Chrome 的 WebDriver
    driver.get('https://www.thsrc.com.tw/')
    accpet_button = driver.find_element(By.CSS_SELECTOR, 'button.swal2-confirm.swal2-styled')
    ac = ActionChains(driver)
    human_like_pause()
    ac.move_to_element(accpet_button)
    ac.click(accpet_button).perform()
    human_like_pause()

    orderWindow = driver.find_element(By.CSS_SELECTOR, 'li.nav-item > a#a_irs')
    ac.move_to_element(orderWindow)
    human_like_pause()
    ac.click(orderWindow)
    ac.perform()

def intoOrderWeb():
    # start_staion = driver.find_element(By.CSS_SELECTOR, 'select[name=selectStartStation][id=select_location01]')
    iframe = driver.find_element(By.CSS_SELECTOR, 'iframe#irsUrl')
    iframeSrc = iframe.get_attribute('src')
    driver.get(f"{iframeSrc}")

def startStaion():
    start_station = driver.find_element(By.CSS_SELECTOR, 'select#select_location01')
    start_sta = Select(start_station)
    start_sta.select_by_value("1")

def arriveStation():
    arrive_station = driver.find_element(By.CSS_SELECTOR, 'select#select_location02')
    start_sta = Select(arrive_station)
    start_sta.select_by_value("12")

def goDate():
    go_date = driver.find_element(By.CSS_SELECTOR, 'input#Departdate02')
    # go_date.clear()
    go_date.send_keys(Keys.CONTROL, "a")
    go_date.send_keys('2025/05/31')

def goTime():
    go_time = driver.find_element(By.CSS_SELECTOR, 'input#toPortalTimeTable')
    # go_date.clear()
    go_time.send_keys(Keys.CONTROL, "a")
    go_time.send_keys('18:00')   

def securityCode():
    #抓取識別碼圖片
    securityCodeImg = driver.find_element(By.CSS_SELECTOR, "img#BookingS1Form_homeCaptcha_passCode")
    #截圖識別碼
    img_bytes = securityCodeImg.screenshot_as_png
    #利用ddddocr來抓取圖片內的英數字
    ocr = ddddocr.DdddOcr()
    result = ocr.classification(img_bytes)
    #輸入識別碼
    securityCode = driver.find_element(By.CSS_SELECTOR, "input#securityCode")
    securityCode.clear()
    securityCode.send_keys(result)

def search():
    searchBtn = driver.find_element(By.CSS_SELECTOR, 'input#SubmitButton')
    ac = ActionChains(driver)
    ac.move_to_element(searchBtn)
    human_like_pause()
    ac.click(searchBtn)
    ac.perform()

def checkTime():
    windows = driver.window_handles
    driver.switch_to.window(windows[-1])
    human_like_pause()
    checkBtn = driver.find_element(By.CSS_SELECTOR, 'input[name=SubmitButton]')
    ac = ActionChains(driver)
    ac.move_to_element(checkBtn)
    human_like_pause()
    ac.click(checkBtn).perform()

def inputInfo():
    input_ID = driver.find_element(By.CSS_SELECTOR, 'input#idNumber')
    input_ID.send_keys("F129878220")
    ac = ActionChains(driver)
    agree_checkbox = driver.find_element(By.CSS_SELECTOR, "input[name=agree]")
    ac.move_to_element(agree_checkbox).click(agree_checkbox).perform()
    human_like_pause()
    # ID_check = driver.find_element(By.CSS_SELECTOR, 'id#BookingS3Form_TicketPassengerInfoInputPanel_passengerDataView_0_passengerDataView2_passengerDataIdNumber')
    # ac.move_to_element(ID_check)
    # human_like_pause()
    # ac.click(ID_check).perform()
    final_submit = driver.find_element(By.CSS_SELECTOR, 'input#isSubmit')
    ac.move_to_element(final_submit).click(final_submit).perform()

def finish_order_info():
    notice_text = driver.find_element(By.CSS_SELECTOR, 'div.alert-body')
    print(notice_text.text)
    tbody = driver.find_element(By.CSS_SELECTOR, 'tbody')
    rows = tbody.find_elements(By.CSS_SELECTOR, 'tr')
    orderInfo = {}
    for row in rows:
        cells = row.find_elements(By.CSS_SELECTOR, 'td')
        if len(cells) == 2:
            key = cells[0].text.strip()
            value = cells[1].text.strip()
            orderInfo[key] = value

# 詳細資訊
    
    info_card = driver.find_element(By.CSS_SELECTOR, 'div.ticket-card')
    howGo = info_card.find_element(By.CSS_SELECTOR, 'span.direction').text.strip()
    date = info_card.find_element(By.CSS_SELECTOR, 'span.date').text.strip()
    train_number = info_card.find_element(By.CSS_SELECTOR, 'p.train-no').text.strip()
    dep_time = info_card.find_element(By.CSS_SELECTOR, '#setTrainDeparture0').text.strip()
    dep_stn = info_card.find_element(By.CSS_SELECTOR, '.departure-stn span').text.strip()
    arr_time = info_card.find_element(By.CSS_SELECTOR, '#setTrainArrival0').text.strip()
    arr_stn = info_card.find_element(By.CSS_SELECTOR, '.arrival-stn span').text.strip()
    duration = info_card.find_element(By.CSS_SELECTOR, '#InfoEstimatedTime0').text.strip()
    seat = driver.find_element(By.CSS_SELECTOR, 'div.seat-label').text.strip()
    ticket_type = driver.find_element(By.CSS_SELECTOR, 'span.uk-leader-fill').text.split()
    ticket_price = driver.find_element(By.CSS_SELECTOR,'div#InfoPrice0').text.split()
    
    
    # ✅ 格式化輸出
    print("=====訂票資訊=====")
    print(orderInfo)
    print("*"*30)
    print(f"【{howGo}】 {date} - 車次 {train_number}")
    print(f"出發：{dep_stn} {dep_time}")
    print(f"抵達：{arr_stn} {arr_time}")
    print(f"行車時間：{duration}")
    print(f"座位：{seat}")
    print(f"{ticket_type} - {ticket_price}")

In [ ]:
for attempt in range(1, 4):
    print(f"第 {attempt} 次嘗試訂票...")
    try:
        openWebsite()
        intoOrderWeb()
        startStaion()
        arriveStation()
        goDate()
        goTime()
        securityCode()
        search()
        checkTime()
        inputInfo()
        finish_order_info()
        break
    except Exception as e:
        print(f"錯誤發生：{e}")
        driver.quit()
        if attempt == 3:
            print("重試達上限，結束程式。")
        